In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import torch
import bitsandbytes as bnb
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
from transformers.utils import is_accelerate_available, is_bitsandbytes_available

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

bnb_version = bnb.__version__
print(f"bitsandbytes version: {bnb_version}")

try:
    bnb.cuda.is_available()
    print("bitsandbytes can utilize CUDA.")
except AttributeError:
    print("bitsandbytes does not have a cuda availability method.")

CUDA available: False
bitsandbytes version: 0.44.1
bitsandbytes does not have a cuda availability method.


In [3]:
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

In [4]:
model_name = "meta-llama/Meta-Llama-3-8B"

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
tokenizer =  AutoTokenizer.from_pretrained(model_name,
                                           token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN,
)